In [4]:
import findspark
findspark.init()
import pyspark

# Big Data Programming 
# Lab 2 - Spark 


import math
from scipy.spatial import distance
from math import sqrt
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
#from pyspark.sql.functions import lit
#from pyspark.sql.functions import min

from pyspark.sql import Row
#from pyspark.sql.types import ArrayType, IntegerType
from pyspark.sql.functions import udf, col , min

from scipy import spatial
from treelib import Tree


# ---------- helper functions ----------    




# ----------------------------------------------
# ----------------- Main Program ---------------

#spark = SparkSession.builder.appName("NBA-kmeans").getOrCreate()
#sc=spark.sparkContext

#--error timeout
import pyspark as spark
#conf = spark.SparkConf().setAppName("applicaiton").set("spark.executor.heartbeatInterval", "200000").set("spark.network.timeout", "300000")
#sc = spark.SparkContext.getOrCreate(conf)
#sqlcontext = spark.SQLContext(sc)

spark = SparkSession.builder.appName("SQL_DataFrame")\
  .master("local")\
  .config("spark.network.timeout", "90000")\
  .config("spark.executor.heartbeatInterval", "80000")\
  .getOrCreate()
sc=spark.sparkContext

#import pyspark as spark
#conf = spark.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
#conf.set("spark.executor.heartbeatInterval","3600s")
#conf.set("spark.executor.heartbeatInterval","120s")
#sc = spark.SparkContext('local[2]', '', conf=conf) # uses 4 cores on your local machine
#---
# ---- load and preprocessing -------

df = spark.read.format("csv").load("./shot_logs.csv", header = "true", inferSchema = "true")
##dataPts = df.filter(df.player_name == 'james harden').select('SHOT_DIST','CLOSE_DEF_DIST').na.drop()
##dataRDD = dataPts.rdd.map(lambda r: (r[0], r[1], r[2]))

#*************** count point 
#dataPts = df.filter(df.player_name == 'james harden').select('SHOT_DIST','CLOSE_DEF_DIST').na.drop()#count points:1054 point
#dataPts = df.select('SHOT_DIST','CLOSE_DEF_DIST').na.drop() #count points: 128069 point
#coords = dataPts.rdd.map(lambda r: (r[0], r[1])).collect()  
coords = [(2, 4),(4, 5), (3,8), (6,7), (1,5), (6,2), (3,7), (8,9), (5,8), (3,6)]
#***************

#dataRDD = dataPts.rdd.map(lambda r: (r[0], r[1]))
##print(dataRDD)
#print(dataRDD.take(dataRDD.count()))
#print(dataRDD.take(5))


dataRDD = sc.parallelize(coords)
#count_point=len(coords)

#rdd = rdd.map(lambda x: [float(i) for i in x])
#print(dataRDD.take(1))
count_point=len(dataRDD.take(dataRDD.count()))
print("count point: ",len(dataRDD.take(dataRDD.count())))
print("points: ",dataRDD.take(dataRDD.count())) #all 
# -------- initialization ---------


rdd_length = dataRDD.map(lambda x: len(x))
rdd_length.collect()
#print(rdd_length.take(rdd_length.count()))
len_vec=rdd_length.take(1)[0] #len demention(features)
#count_point=len(coords)
#print(len_vec)


totalindex=0
ir=0.2
index=0
iters = 0 
#old_centroid = int_centroid
#old_centroid = coords
k=3
run=1

#list_index_nearset=list(range(0, count_point) )
list_index_nearset=[0]*count_point
#print("list_index_nearset",list_index_nearset)

for m in range(run):

    
    schema = StructType([StructField('id', IntegerType(), True)])
    df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema)
    df = spark.range(0, count_point).withColumnRenamed("id", "idx_point")
    df.show()


    df= df.withColumnRenamed('_1','idx_point')

    #df8 = df.rdd.map(lambda x: [r for r in row(x)]).collect() 
    #print(df8)
    
    #from pyspark.ml.feature import VectorAssembler
    #FEATURES_COL = df.columns[1:]
    ##print(FEATURES_COL)
    #vecAssembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
    #df = vecAssembler.transform(df).select('idx_point', "features")
    df= df.withColumn('ind_nxt_point', F.lit(0))


    df.show()

    #def make_list(col):
        #return list(map(int,[x for x in range(col+1) if x % 2 == 0]))    
    #make_list_udf = udf(make_list, ArrayType(IntegerType()))
    #df2 = df.withColumn('features',make_list_udf(col('features')))    
    
    ls_clu_Pnt=[] #list E:N,N,N..
    ls_sel_pnt=[] #initial points & clustred points
    ls_clu_Pnt = df.select('idx_point').rdd.flatMap(lambda x: [x]).collect()  #list E:N,N,N..
    ls_clu_Pnt= [ele.__getattr__('idx_point') for ele in ls_clu_Pnt] #convert row to one list
    ls_clu_Pnt= [ls_clu_Pnt[i:i+1] for i in range(0, len(ls_clu_Pnt), 1)] #convert list 1d to list 2d
    #print('ls_clu_Pnt: ',ls_clu_Pnt.__str__()) # all element print

    initial_Point = list(map(lambda row: row[0], df.select('idx_point').rdd.takeSample(False, 3))) #convert row out to one number
    print('initial_Point: ',initial_Point)
    #points_in_cluster = initial_Point
    #ls_sel_pnt= initial_Point #list: all selected points for clustering(selected point in all clutser)
    ls_sel_pnt.append(initial_Point[0])
    ls_sel_pnt.append(initial_Point[1])
    ls_sel_pnt.append(initial_Point[2])
    
    ls_clu_Pnt[ls_sel_pnt[0]].append(-1)
    ls_clu_Pnt[ls_sel_pnt[1]].append(-1)
    ls_clu_Pnt[ls_sel_pnt[2]].append(-1)
    print('ls_clu_Pnt: ',ls_clu_Pnt.__str__()) # all element print
    print('ls_sel_pnt: ',ls_sel_pnt)
    #df = spark.createDataFrame(points_in_cluster, IntegerType()) #convert list to df
    #df2 = spark.createDataFrame([("Range 1", list([101,105])), ("Range 2", list([200, 203]))])

    #----------------part 2-----------
    print("*****************start clustering********************")
          
    

    
    tree = spatial.cKDTree(coords,leafsize=16)
    kdt_b = sc.broadcast(tree)
    #distance,index = kdt_b.value.query([2.4, 4], k=2)
    #print(distance,index)    
    #print(index[1])    



count point:  10
points:  [(2, 4), (4, 5), (3, 8), (6, 7), (1, 5), (6, 2), (3, 7), (8, 9), (5, 8), (3, 6)]
+---------+
|idx_point|
+---------+
|        0|
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
+---------+

+---------+-------------+
|idx_point|ind_nxt_point|
+---------+-------------+
|        0|            0|
|        1|            0|
|        2|            0|
|        3|            0|
|        4|            0|
|        5|            0|
|        6|            0|
|        7|            0|
|        8|            0|
|        9|            0|
+---------+-------------+

initial_Point:  [3, 8, 0]
ls_clu_Pnt:  [[0, -1], [1], [2], [3, -1], [4], [5], [6], [7], [8, -1], [9]]
ls_sel_pnt:  [3, 8, 0]
*****************start clustering********************


In [5]:
    #---990923
    def deep_index(lst, w , frs_sec_idx, end_idx): #find num index in cluster(ls_clu_Pnt[1:] ,for two father)
        return [(i, sub.index(w)) for (i, sub) in enumerate(lst) if w in sub[frs_sec_idx:end_idx]] #[1:] =>num elemnt of witch cluster?
    #def deep_index2(lst, w): #find num index in cluster(ls_clu_Pnt[1:] ,for two father)
    #    return [(i, sub.index(w)) for (i, sub) in enumerate(lst) if w in sub[0:1]] 


    def clu_update(vec,num1):
        tmp_point= vec[0]
        if tmp_point==elec_point and (num1 not in vec):
            vec.append(num1)       
        #if (num1 in ls_sel_pnt):
            #l=[[0], [1], [2], [3, 8], [4.0], [5], [6], [7, 3, 8], [8, 4], [9]]
            ##print('l: ',l.__str__())
            #Asign_point= deep_index(l, 3)
            #Asign_point= deep_index(ls_clu_Pnt, num1) #find num index in cluster(ls_clu_Pnt[1:] ,for two father)
            ##print(Asign_point) =>[(7, 1)] (1->index 3)
            #Asign_point=Asign_point[0][0]
            ##print(Asign_point) => 7 (7 -> cluster dad(Asign))

            #user_func = udf (lambda x,y: [i for i, e in enumerate(x) if e==y ]) #find num index in list in column features
            #df_idx_dad_min_dis = df.filter((df['idx_point'] == Asign_point) | (df['idx_point'] == elec_point))
            #df_idx_dad_min_dis = df.filter(df['idx_point'] == 7) 
            #df5=df
            #    .select(df['idx_point'], df['features'])\
            #    .withColumn('item_position',user_func(df.features,F.lit(num1)))\
            #    .orderBy("item_position", ascending=True).limit(1).collect()
            #idx_dad_min_dis = df_idx_dad_min_dis[0][0]
            ##print(idx_dad_min_dis[0][0])       
            #vec[idx_dad_min_dis:-1].append(num1) 
        return vec       

    def indexnext(idx_point1, ind_nxt_point1):
        d=ind_nxt_point1
        if idx_point1 == elec_point: 
            if (d<count_point):
                d= d+1
            else: d=0    
        return d
    
    def update_father_child(x,tmp_next_Points,father_point):
        if ((len(x)>2) and (tmp_next_Points in x[1:2])):
            x[1]=father_point
        return x

                        
    def child_other_father(x1,list_num_childs1):
        if (len(x1)>2):
            if x1[0] not in list_num_childs1:  
                return (x1[0]) 
        return -1
    

    #print('ls_clu_Pnt: ',ls_clu_Pnt.__str__()) # all element #print
    #print('ls_sel_pnt: ',ls_sel_pnt)

    #totoalindex=round(ir*(count_point * count_point),0)
    ir=0.2
    totoalindex=round(ir*(count_point * count_point),0)
    #print("totoalindex:", totoalindex)
    
    index=0
    i=1
    while (len(ls_sel_pnt)< count_point) and (index<totoalindex):
    #while i<=100:
    #while i==1:
        rdd_sel_pnt=sc.parallelize(ls_sel_pnt)
        #elec_point =  sc.parallelize(ls_sel_pnt).takeSample(False, 1)
        elec_point =  rdd_sel_pnt.takeSample(False, 1)
        elec_point= elec_point[0]
        rdd_cluster_pnt = sc.parallelize(ls_clu_Pnt)
        #elec_point=0

        
        
        index=index+1
        print("-------------------------------------------index: ",index)
        #print('elec_point: ',elec_point)
        
        
        #filter_udf = udf(indexnext, IntegerType())
        #df= df.withColumn("ind_nxt_point", filter_udf(df['idx_point'], df['ind_nxt_point'])) #update index for point next
        #df.show()
        #df = df.filter(col("idx_point").between(0, 0)).withColumn('ind_nxt_point',F.when(F.col('idx_point') == elec_point, 55).otherwise(88))
        #tmp_index_nearset = df.filter(df['idx_point'] == elec_point).collect()[0][1] #+1:first point is self
        #df.show()
        
        list_index_nearset[elec_point]=list_index_nearset[elec_point]+1
        ##print("list_index_nearset",list_index_nearset)
        tmp_index_nearset=list_index_nearset[elec_point]
        
        
        ##print("tmp_index_nearset: ",tmp_index_nearset)
        
        

        distance,index_dis = kdt_b.value.query(coords[elec_point], k=tmp_index_nearset+1)#+1:first point is self
        #distance,index_dis = kdt_b.value.query(coords[elec_point], k=7)
        ##print(distance,index_dis)  
        ##print("vec_dis:",index_dis)  
        #tmp_next_Points=coords[index_dis[tmp_index_nearset]]
        tmp_next_Points=index_dis[tmp_index_nearset]
        ##print(index_dis[tmp_index_nearset])        
        #print('tmp_next_Points: ',tmp_next_Points)


       

        #rdd= sc.parallelize(ls_clu_Pnt)
        #ls_clu_Pnt = rdd.map(lambda x: clu_update(x,tmp_next_Points)).collect()
        if  tmp_next_Points in ls_sel_pnt: #N befor selected
            #----
            #if (elec_point in ls_sel_pnt) and (len(ls_sel_pnt)==3): continue #run 1 and two point are in initial_point
            #Asign_point= deep_index(ls_clu_Pnt, elec_point, 0,1)[0][0] # jsut col 0 returned for check
            if  elec_point not in ls_clu_Pnt[tmp_next_Points][2:]: #check (4,0) for (0,4) ,check else col 2 is father

                
                #rdd = sc.parallelize(ls_clu_Pnt)#find elect father (check for e & n are in samecluster(no assignment))
                rdd = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: (len(x)>2) and (elec_point in x[2:])).collect()
                if (len(rdd) >0): 
                    if rdd[0][1] == -1: father_elect= rdd[0][0]
                    else:  father_elect= rdd[0][1]
                else:  father_elect=elec_point
                #rdd = sc.parallelize(ls_clu_Pnt)#find tmp_next_Points father (check for e & n are in samecluster(no assignment))
                rdd = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: (len(x)>2) and (tmp_next_Points in x[2:])).collect()
                if (len(rdd) >0): 
                    if rdd[0][1] == -1: father_tmp_next_Points= rdd[0][0]
                    else:  father_tmp_next_Points= rdd[0][1]
                else:  father_tmp_next_Points=tmp_next_Points
                    
                
                #rdd = sc.parallelize(ls_clu_Pnt)#check for e & n are in samecluster(no assignment) 
                #rdd = rdd.map(lambda x: x).filter(lambda x: (len(x)>2 and (elec_point in x[2:]) and (tmp_next_Points2 in x[2:])) ).collect()
                #rdd = rdd.map(lambda x: x).filter(lambda x: (len(x)>2).filter(lambda x: elec_point in x[2:] and (tmp_next_Points2 in x[2:])) ).collect()
                #if (len(rdd) ==0):
                if (father_elect != father_tmp_next_Points):
                
                    Asign_point= deep_index(ls_clu_Pnt, tmp_next_Points, 2,len(ls_clu_Pnt)) #find num index in cluster(ls_clu_Pnt[2:] ,for two father)
                    #print('Asign_point(befor father): ',Asign_point) #=>[(7, 1)] (7 -> col 0(father) , 1 -> index tmp_next_Points in [])
                
                    if len(Asign_point)>0: #N befor selected and is not one of initial point
                        #print("N befor selected and is not one of initial point")
                        father_befor=Asign_point[0][0]
                        father_new= elec_point  
                        #point_father = [coords[father_befor],coords[father_new]]
                        tree_father = spatial.cKDTree([coords[father_befor],coords[father_new]],leafsize=16)
                        distance,index_dis = tree_father.query(coords[tmp_next_Points])
                        ##print(distance,index_dis)
                        if (index_dis==0): 
                            idx_dad_min_dis=father_befor 
                            idx_dad_max_dis=father_new 
                        else: 
                            idx_dad_min_dis=father_new 
                            idx_dad_max_dis=father_befor 
                        #print("idx_dad_min_dis: ",idx_dad_min_dis)
                        
                        if (tmp_next_Points) not in ls_clu_Pnt[idx_dad_min_dis][2:]:
                            #rdd = sc.parallelize(ls_clu_Pnt)
                            rdd = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: (len(x)>2 and (elec_point in x[2:])) ).collect() #find index father (in col 2 of list)
                            if (len(rdd) >0):
                                if rdd[0][1] == -1: 
                                    father_point= rdd[0][0]
                                else:
                                    father_point= rdd[0][1]
                                ls_clu_Pnt[elec_point].append(father_point)  #(col1:point number , col2: father point, col3: N ...)     
                            else:
                                if (len(ls_clu_Pnt[idx_dad_min_dis])==1):                
                                    ls_clu_Pnt[idx_dad_min_dis].insert(1,-1)
                            #print("end 5")
                            ls_clu_Pnt[idx_dad_min_dis].append(tmp_next_Points) 
                            if (tmp_next_Points in ls_clu_Pnt[idx_dad_max_dis][2:]):
                                ls_clu_Pnt[idx_dad_max_dis].remove(tmp_next_Points) 
                                if ((len(ls_clu_Pnt[idx_dad_max_dis])==2) and (ls_clu_Pnt[idx_dad_max_dis][1] !=-1)):
                                    ls_clu_Pnt[idx_dad_max_dis].remove(ls_clu_Pnt[idx_dad_max_dis][1]) #remove child => len=2 => remve childa's father
                            #**************---update childs(find childs & assign other father(elected)    
                            #--find elected's father 
                            #rdd = sc.parallelize(ls_clu_Pnt)
                            rdd = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: (len(x)>2 and (elec_point in x[2:])) ).collect() #find index father (in col 2 of list)
                            if (len(rdd) >0):
                                if rdd[0][1] == -1: 
                                    father_point= rdd[0][0]
                                else:
                                    father_point= rdd[0][1]
                                #if (len(ls_clu_Pnt[elec_point])==1):
                                    #ls_clu_Pnt[elec_point].insert(1,father_point) #(col1:point number , col2: father point, col3: N ...) 
                                #ls_clu_Pnt[tmp_next_Points][1]=father_point
                            else:
                                #if (elec_point not in initial_Point):
                                #ls_clu_Pnt[elec_point].insert(1,-1)
                                father_point=-1
                            #--
                            if (len(ls_clu_Pnt[tmp_next_Points])>2):
                                #--find list childs of childs tmp_next_Points & update fathers
                                #print("father point:",father_point)
                                ls_clu_Pnt = rdd_cluster_pnt.map(lambda x: update_father_child(x,tmp_next_Points,father_point)).collect()#[1:2] searech all items with index[1] only
                                
                                #rdd = sc.parallelize(ls_clu_Pnt)
                                #list_childs = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: ((len(x)>2) and (tmp_next_Points in x[1:2])) ).collect()#[1:2] searech all items with index[1] only
                                ##print('list_childs of childs ',tmp_next_Points,': ', list_childs)
                                #if (len(list_childs) >0):
                                #    for x in list_childs: x[1]= father_point
                                #--
                                #--remove (befor electe) and reasign new elected
                                if (ls_clu_Pnt[tmp_next_Points][1]==-1):
                                    initial_Point.remove(tmp_next_Points)
                                    elec_point_again =  sc.parallelize(list(set(list(range(len(ls_clu_Pnt))))-set(ls_sel_pnt))).takeSample(False, 1) # reasignment
                                    elec_point_again= elec_point_again[0]
                                    #print('elec_point_again: ',elec_point_again)
                                    ls_clu_Pnt[elec_point_again].insert(1,-1)
                                    ls_sel_pnt.append(elec_point_again)
                                    initial_Point.append(elec_point_again)  
                                    #print('initial_Point new:', initial_Point)                            
                                #--
                            #if (len(ls_clu_Pnt[elec_point])==1):
                            #    ls_clu_Pnt[elec_point].insert(1,father_point) #(col1:point number , col2: father point, col3: N ...) 
                            #ls_clu_Pnt[elec_point].append(tmp_next_Points)  
                            #if (len(ls_clu_Pnt[tmp_next_Points])>2):
                            #    ls_clu_Pnt[tmp_next_Points][1]=father_point
                            #***************------   
                            
                            #----update childs(find childs & assign to other elected)    
                            ##def child_other_father(x1,list_num_childs1):
                            ##    if (len(x1)>2):
                            ##        if x1[0] not in list_num_childs1:  
                            ##            return (x1[0]) 
                            ##    return -1
                            #if len(ls_clu_Pnt[tmp_next_Points])>2:
                            #    idx_cluster_child= ls_clu_Pnt[tmp_next_Points][1]
                            #    #print('idx_cluster_child: ',idx_cluster_child)
                            #    rdd2 = sc.parallelize(ls_clu_Pnt)#select all points of cluster father(col2)-> find child_other_father and remove them
                            #    list_samecluster_full_childs = rdd2.map(lambda x: x).filter(lambda x: ((len(x)>2) and (idx_cluster_child in x[1:2]) )).collect() #find index father (in col 2 of list)
                            #    #print('list_samecluster_full_childs:' , list_samecluster_full_childs)
                            #    list_num_childs=[]
                            #    for x in list_samecluster_full_childs: list_num_childs.append(x[2:]) #all child's
                            #    l= sum(list_num_childs, []) #convert [[4,5],[3]] to [4,5,3]
                            #    #print('list_num_childs: ',list_num_childs)
                            #    rdd = sc.parallelize(list_samecluster_full_childs)
                            #    list_child_other_father = rdd.map(lambda x: child_other_father(x, list_num_childs)).collect()
                            #    #print('list_child_other_father: ',list_child_other_father)
                            #    while -1 in list_child_other_father:  list_child_other_father.remove(-1)
                            #    list_child_other_father.remove(tmp_next_Points) # father all child's
                            #    #print('list_child_other_father: ',list_child_other_father)
                            #    list_childs = list(filter(lambda list_samecluster_full_childs: list_samecluster_full_childs[0] not in list_child_other_father, list_samecluster_full_childs))
                            #    #print('list_childs ',idx_cluster_child,': ', list_childs)
                            #    for x in list_childs: x[1]=elec_point
                            #    #print('list_childs ',idx_cluster_child,' (update): ', list_childs)
                            #-----------------
        

                            
                    else: # N is one of initial point
                        if (len(ls_clu_Pnt[tmp_next_Points])<3): #N subset initial_Point & befor not selected(dont have child)
                            #print('initial_Point:', initial_Point)
                            #if (len(ls_clu_Pnt[elec_point])==1):                
                            #    ls_clu_Pnt[elec_point].insert(1,-1)
                            #--find elected's father 
                            #rdd = sc.parallelize(ls_clu_Pnt)
                            rdd = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: (len(x)>2 and (elec_point in x[2:])) ).collect() #find index father (in col 2 of list)
                            if (len(rdd) >0):
                                if rdd[0][1] == -1: 
                                    father_point= rdd[0][0]
                                else:
                                    father_point= rdd[0][1]
                                if (len(ls_clu_Pnt[elec_point])==1):
                                    #ls_clu_Pnt[elec_point].append(father_point)  #(col1:point number , col2: father point, col3: N ...) 
                                    ls_clu_Pnt[elec_point].insert(1,father_point)
                            #--
                            ls_clu_Pnt[elec_point].append(tmp_next_Points)  
                            ls_clu_Pnt[tmp_next_Points].remove(-1) 
                            initial_Point.remove(tmp_next_Points)
                            elec_point_again =  sc.parallelize(list(set(list(range(len(ls_clu_Pnt))))-set(ls_sel_pnt))).takeSample(False, 1) # reasignment
                            elec_point_again= elec_point_again[0]
                            #print('elec_point_again: ',elec_point_again)
                            ls_clu_Pnt[elec_point_again].insert(1,-1)
                            ls_sel_pnt.append(elec_point_again)
                            initial_Point.append(elec_point_again)  
                            #print('initial_Point new:', initial_Point)
                        
                        else: #((len(ls_clu_Pnt[tmp_next_Points])<3): #N subset initial_Point & befor..)
                        #N(tmp_next_Points) subset initial_Point & befor selected(have child)
                            #print('N(tmp_next_Points) subset initial_Point & befor selected(have child)')
                            #---------------update childs(find childs & assign other father(elected)    
                            #--find elected father 
                            #rdd = sc.parallelize(ls_clu_Pnt)
                            rdd = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: (len(x)>2 and (elec_point in x[2:])) ).collect() #find index father (in col 2 of list)
                            father_point=-100 # for know of wrong
                            if (len(rdd) >0): # e sure have father
                                if rdd[0][1] == -1: 
                                    father_point= rdd[0][0]
                                else:
                                    father_point= rdd[0][1]
                                #if (len(ls_clu_Pnt[elec_point])==1):
                                    #ls_clu_Pnt[elec_point].insert(1,father_point) #(col1:point number , col2: father point, col3: N ...) 
                                #ls_clu_Pnt[tmp_next_Points][1]=father_point
                            else: # e is elected
                                #rdd = rdd.map(lambda x: x).filter(lambda x: (len(x)>2 and (elec_point in x[0:1])) ).collect() #find index father (in col 2 of list)
                                #if (len(ls_clu_Pnt[elec_point])) ==2):# e is elected
                                #    father_point=ls_clu_Pnt[tmp_next_Points][1]
                                #else:
                                    #father_point=ls_clu_Pnt[tmp_next_Points][1]
                                father_point=elec_point
                                    
                            #--
                            if (len(ls_clu_Pnt[tmp_next_Points])>2):
                                #--find list childs of childs tmp_next_Points & update fathers
                                #print("father point:",father_point)
                                ls_clu_Pnt = rdd_cluster_pnt.map(lambda x: update_father_child(x,tmp_next_Points,father_point)).collect()#[1:2] searech all items with index[1] only

                                #rdd = sc.parallelize(ls_clu_Pnt)
                                #list_childs = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: ((len(x)>2) and (tmp_next_Points in x[1:2])) ).collect()#[1:2] searech all items with index[1] only
                                ##print('list_childs of childs ',tmp_next_Points,': ', list_childs)
                                ##print("father point:",father_point)
                                #if (len(list_childs) >0):
                                #    for x in list_childs: 
                                #        #print("child:",x[1])
                                #        x[1]= father_point
                                #        #print("update child father:",x[1])
                                #--
                                #--remove (befor electe) and reasign new elected
                                if (ls_clu_Pnt[tmp_next_Points][1]==-1):
                                    initial_Point.remove(tmp_next_Points)
                                    elec_point_again =  sc.parallelize(list(set(list(range(len(ls_clu_Pnt))))-set(ls_sel_pnt))).takeSample(False, 1) # reasignment
                                    elec_point_again= elec_point_again[0]
                                    #print('elec_point_again: ',elec_point_again)
                                    ls_clu_Pnt[elec_point_again].insert(1,-1)
                                    ls_sel_pnt.append(elec_point_again)
                                    initial_Point.append(elec_point_again)  
                                    #print('initial_Point new:', initial_Point)                            
                                #--
                            if (len(ls_clu_Pnt[elec_point])==1):
                                ls_clu_Pnt[elec_point].insert(1,father_point) #(col1:point number , col2: father point, col3: N ...) 
                            ls_clu_Pnt[elec_point].append(tmp_next_Points)  
                            if (len(ls_clu_Pnt[tmp_next_Points])>2):
                                ls_clu_Pnt[tmp_next_Points][1]=father_point
                            #-----------------   
                #else:   #(len(rdd) ==0)                 
                    #print("elec_point & tmp_next_Points are in same cluster")
                    
            else: # (elec_point not in ls_clu_Pnt[tmp_next_Points][2:]: #check (4,0) for (0,4))
                #if (elec_point in ls_sel_pnt) and (len(ls_sel_pnt)==3):  #run 1 and two point are in initial_point
                if (len(ls_sel_pnt)==3):  #run 1 and two point are in initial_point
                    #print('elec_point & tmp_next_Points are in initial_Point for first run')
                    ls_clu_Pnt[elec_point].append(tmp_next_Points)  
                    ls_clu_Pnt[tmp_next_Points].remove(-1) 
                    initial_Point.remove(tmp_next_Points)
                    elec_point_again =  sc.parallelize(list(set(list(range(len(ls_clu_Pnt))))-set(ls_sel_pnt))).takeSample(False, 1) # reasignment
                    elec_point_again= elec_point_again[0]
                    #print('elec_point_again: ',elec_point_again)
                    ls_clu_Pnt[elec_point_again].insert(1,-1)
                    ls_sel_pnt.append(elec_point_again)
                    initial_Point.append(elec_point_again)  
                    #print('initial_Point new:', initial_Point)
                #else: #((elec_point in ls_sel_pnt) and (len(ls_sel_pnt)==3):)
                    #print('2,1 => 1,2 : n father e => e father n')
        else: #(tmp_next_Points in ls_sel_pnt: #N befor selected)
            #Asign_point= deep_index2(ls_clu_Pnt, tmp_next_Points) #find num index in cluster(ls_clu_Pnt[1:] ,for two father)
            ##print(Asign_point) #=>[(7, 1)] (1->index 3)
            #Asign_point=Asign_point[0][0]
            ##print(Asign_point) #=> 7 (7 -> cluster dad(Asign))            
            #print("tmp_next_Points without father & child")
            #rdd = sc.parallelize(ls_clu_Pnt)
            #rdd = rdd.map(lambda x: x).filter(lambda x: (len(x)>1 and (x[0]==tmp_next_Points)  and (x[1]!=-1))).collect() #find index father (in col 2 of list)
            #rdd = rdd.map(lambda x: x).filter(lambda x: (len(x)>2 and (elec_point in x[2:])) ).collect() #find index father (in col 2 of list)
            rdd = rdd_cluster_pnt.map(lambda x: x).filter(lambda x: (len(x)>0 and (elec_point in x[2:])) ).collect() #find index father (in col 2 of list)
            if (len(rdd) >0):
                if rdd[0][1] == -1: 
                    father_point= rdd[0][0]
                else:
                    father_point= rdd[0][1]
                if (len(ls_clu_Pnt[elec_point])==1):
                    ls_clu_Pnt[elec_point].append(father_point)  #(col1:point number , col2: father point, col3: N ...) 
            else:
                if (elec_point not in initial_Point):
                    ls_clu_Pnt[elec_point].insert(1,-1)
                    
            ls_clu_Pnt[elec_point].append(tmp_next_Points)       
            ls_sel_pnt.append(tmp_next_Points)
            
            

        #print('ls_clu_Pnt: ',ls_clu_Pnt.__str__())
        #print('ls_sel_pnt: ',ls_sel_pnt.__str__())
        i=i+1
            
    
    #print("-----------#print clusters-------------")
    rdd = sc.parallelize(ls_clu_Pnt)#find elect father (check for e & n are in samecluster(no assignment))
    list_all_cluster = rdd.map(lambda x: x).filter(lambda x: (len(x)>1) and (x[1]==-1)).collect()
    ##print(list_all_cluster7)
    list_all_full_cluster=[]
    if (len(list_all_cluster) >0): 
        for y in list_all_cluster: 
            ##print(y)
            list_cluster=[]
            list_cluster = rdd.map(lambda x: x).filter(lambda x: (len(x)>1) and (y[0]==x[1])).collect()
            list_cluster.insert(0,y)
            ##print(list_cluster)
            for x in list_cluster: x.remove(x[1])
            list_cluster= sum(list_cluster, []) #convert [[4,5],[3]] to [4,5,3]
            ##print(list_cluster)
            list_cluster = list( dict.fromkeys(list_cluster) )
            ##print(list_cluster)
            list_all_full_cluster.append(list_cluster)
    
    print("list_all_full_cluster:",list_all_full_cluster)
        

#sc.stop()


        


-------------------------------------------index:  1
-------------------------------------------index:  2
-------------------------------------------index:  3
-------------------------------------------index:  4
-------------------------------------------index:  5
-------------------------------------------index:  6
-------------------------------------------index:  7
-------------------------------------------index:  8
-------------------------------------------index:  9
-------------------------------------------index:  10
list_all_full_cluster: [[0, 4, 1, 6, 9], [3, 8, 7, 2], [5]]
